In [1]:
import dill

#### Notes on file structure

- results are stored in ../results/results
- figures are stored in ../results/figures
- folder structure is based on data (i.e. start/end point, number of datapoints and noise level)
- file names are based on the actual model i.e. system name, l1 and l2 values 
- file names are separated by "_"
- relevant results files are ...
    - f"{filename_addendum}_MLL.pkl"
    - f"{filename_addendum}_MLL_logs.pkl"
    - f"{filename_addendum}_MAP.pkl"
    - f"{filename_addendum}_MAP_logs.pkl"
    - f"{filename_addendum}_mean_ode_satisfaction_error_MLL.pkl"
    - f"{filename_addendum}_mean_ode_satisfaction_error_MAP.pkl"
    - f"{filename_addendum}_sample_ode_satisfaction_error_MLL.pkl"
    - f"{filename_addendum}_sample_ode_satisfaction_error_MAP.pkl"
    - f"{filename_addendum}_MLL_model_train_MSEs.pkl"
    - f"{filename_addendum}_MAP_model_train_MSEs.pkl"
    - f"{filename_addendum}_MLL_model_test_MSEs.pkl"
    - f"{filename_addendum}_MAP_model_test_MSEs.pkl"
- relevant figure files are ...
    - f"MLL_model_posterior_{system_name}_l1-{l1_param_val}_l2-{l2_param_val}_{START}-{END}-{COUNT}_{noise_level}.png"
    - f"MAP_model_posterior_{system_name}_l1-{l1_param_val}_l2-{l2_param_val}_{START}-{END}-{COUNT}_{noise_level}.png"

#### Notes on relevant metrics / summaries / statistics
-  